In [1]:
import numpy as np
import pandas as pd
import datetime

# Загрузка старых

In [2]:
old_table = pd.read_csv('/kaggle/input/mmvb-tovar/data.csv')
old_table.set_index(['<TICKER>','<DATE>'],inplace=True)
old_table

<CLOSE>    <VOL>
<TICKER>         <DATE>                      
@Brent Crude Oil 2017-07-17   48.370  22907.0
                 2017-07-18   48.730  24956.0
                 2017-07-19   49.650  23396.0
                 2017-07-20   49.320  23962.0
                 2017-07-21   47.820  26005.0
...                              ...      ...
Silver           2023-01-03   23.963      0.0
                 2023-01-04   23.776      0.0
                 2023-01-05   23.228      0.0
                 2023-01-06   23.827      0.0
                 2023-01-07   23.827      0.0

[17999 rows x 2 columns]

In [3]:
# https://mfd.ru/export/?groupId=2&selectedTickers=1*,2*,3*,4*,170*,1878,1924,1925,1926#Tickers=1*%2C2*%2C3*%2C4*%2C170*%2C1878%2C1924%2C1925%2C1926&Alias=false&Period=7&timeframeValue=1&timeframeDatePart=day&StartDate=06.01.2023&EndDate=06.01.2023&SaveFormat=0&SaveMode=0&FileName=mfdexport_1day_06012023_06012023.txt&FieldSeparator=%2C&DecimalSeparator=.&TimeFormat=HHmmss&DateFormatCustom=yyyy-MM-dd&AddHeader=true&RecordFormat=2&Fill=false

def get_new_table(delta_start=-10,delta_end=0):
    end_date = (datetime.datetime.today() + datetime.timedelta(days=delta_end)).strftime('%d.%m.%Y')
    start_date = (datetime.datetime.today() + datetime.timedelta(days=delta_start)).strftime('%d.%m.%Y')
    url = "https://mfd.ru/export/handler.ashx/mfdexport_1day.txt?TickerGroup=2&Tickers=1*%2C2*%2C3*%2C4*%2C170*%2C1878%2C1924%2C1925%2C1926&Alias=false&Period=7&timeframeValue=1&timeframeDatePart=day&StartDate=" + start_date + "&EndDate=" + end_date + "&SaveFormat=0&SaveMode=0&FileName=mfdexport_1day.txt&FieldSeparator=%253b&DecimalSeparator=.&DateFormat=yyyy-MM-dd&TimeFormat=HHmmss&DateFormatCustom=&TimeFormatCustom=&AddHeader=true&RecordFormat=2&Fill=true"
    table = pd.read_csv(url,sep=";")
    new_table = table[["<TICKER>","<DATE>","<CLOSE>","<VOL>"]]
    new_table.set_index(['<TICKER>','<DATE>'],inplace=True)
    new_table
    return new_table

# get_new_table(delta_start=-30,delta_end=-20)

# Закачка последних

In [4]:
new_table = get_new_table()
new_table

<CLOSE>  <VOL>
<TICKER>     <DATE>                     
Gold         2022-12-28  1804.268      0
             2022-12-29  1814.445      0
             2022-12-30  1823.370      0
             2022-12-31  1823.580      0
             2023-01-01  1823.580      0
...                           ...    ...
@Natural gas 2023-01-03     4.014  31015
             2023-01-04     4.155  26862
             2023-01-05     3.756  29042
             2023-01-06     3.734  28979
             2023-01-07     3.734      0

[99 rows x 2 columns]

In [5]:
concat_table = new_table

# Обновление существующих

In [6]:
old_table.update(new_table)

# Добавление новых

In [7]:
concat_table = pd.concat([old_table,new_table[~new_table.index.isin(old_table.index)]])
concat_table.sort_index(inplace=True)
concat_table

<CLOSE>    <VOL>
<TICKER>         <DATE>                      
@Brent Crude Oil 2017-07-17   48.370  22907.0
                 2017-07-18   48.730  24956.0
                 2017-07-19   49.650  23396.0
                 2017-07-20   49.320  23962.0
                 2017-07-21   47.820  26005.0
...                              ...      ...
Silver           2023-01-03   23.963      0.0
                 2023-01-04   23.776      0.0
                 2023-01-05   23.228      0.0
                 2023-01-06   23.827      0.0
                 2023-01-07   23.827      0.0

[17999 rows x 2 columns]

# Сохранение

In [8]:
concat_table.reset_index(inplace=True)
concat_table

,<TICKER>,<DATE>,<CLOSE>,<VOL>
0,@Brent Crude Oil,2017-07-17,48.370,22907.0
1,@Brent Crude Oil,2017-07-18,48.730,24956.0
2,@Brent Crude Oil,2017-07-19,49.650,23396.0
3,@Brent Crude Oil,2017-07-20,49.320,23962.0
4,@Brent Crude Oil,2017-07-21,47.820,26005.0
...,...,...,...,...
17994,Silver,2023-01-03,23.963,0.0
17995,Silver,2023-01-04,23.776,0.0
17996,Silver,2023-01-05,23.228,0.0
17997,Silver,2023-01-06,23.827,0.0


In [9]:
concat_table.to_csv("data.csv",index=False)

# Подкачка старых

In [10]:
# concat_table = old_table.copy()
# for de in range(1000,2000,20):
#     new_table = get_new_table(delta_start=-de-20,delta_end=-de)
#     concat_table = pd.concat([concat_table,new_table[~new_table.index.isin(concat_table.index)]])
#     print(-de-20,-de)
# #     break
# concat_table.sort_index(inplace=True)
# concat_table.reset_index().to_csv("data.csv",index=False)
# concat_table

# Закрытия

In [11]:
close_table = concat_table.pivot_table(index='<DATE>',columns='<TICKER>',values='<CLOSE>')
close_table.index.name = "Dates"
close_table.reset_index().to_csv('close_table.csv',index=False)
close_table

<TICKER>,@Brent Crude Oil,@Gas Oil,@Heating oil,@Light crude oil,@Natural gas,Gold,Palladium,Platinum,Silver
Dates,,,,,,,,,
2017-07-17,48.37,444.00,1.5026,46.18,3.010,1234.000,865.000,923.000,16.072
2017-07-18,48.73,447.00,1.5142,46.43,3.070,1242.300,864.500,926.690,16.243
2017-07-19,49.65,459.50,1.5559,47.27,3.057,1240.940,858.450,918.250,16.250
2017-07-20,49.32,457.75,1.5500,46.93,3.024,1244.240,846.500,926.780,16.300
2017-07-21,47.82,448.50,1.5184,45.57,2.945,1254.530,844.500,933.310,16.482
...,...,...,...,...,...,...,...,...,...
2023-01-03,82.43,870.75,3.0839,77.21,4.014,1838.140,1704.000,1086.154,23.963
2023-01-04,78.02,844.75,2.9892,73.18,4.155,1855.659,1787.000,1077.000,23.776
2023-01-05,78.68,846.75,2.9755,73.71,3.756,1832.870,1736.500,1060.108,23.228


# Объемы

In [12]:
vol_table = concat_table.pivot_table(index='<DATE>',columns='<TICKER>',values='<VOL>')
vol_table.index.name = "Dates"
vol_table.reset_index().to_csv('vol_table.csv',index=False)
vol_table

<TICKER>,@Brent Crude Oil,@Gas Oil,@Heating oil,@Light crude oil,@Natural gas,Gold,Palladium,Platinum,Silver
Dates,,,,,,,,,
2017-07-17,22907.0,7390.0,18302.0,46830.0,9523.0,0.0,0.0,0.0,0.0
2017-07-18,24956.0,8097.0,20212.0,74039.0,12759.0,0.0,0.0,0.0,0.0
2017-07-19,23396.0,8080.0,17408.0,79390.0,11388.0,0.0,0.0,0.0,0.0
2017-07-20,23962.0,8628.0,20059.0,80296.0,14921.0,0.0,0.0,0.0,0.0
2017-07-21,26005.0,9565.0,20924.0,87190.0,15776.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2023-01-03,43564.0,21523.0,33590.0,63818.0,31015.0,0.0,0.0,0.0,0.0
2023-01-04,44415.0,23092.0,38437.0,64231.0,26862.0,0.0,0.0,0.0,0.0
2023-01-05,41847.0,21898.0,34833.0,61992.0,29042.0,0.0,0.0,0.0,0.0
